In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from transformers import pipeline
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import ollama
import json


In [46]:
df = pd.read_csv("/Users/chandlershortlidge/Desktop/Ironhack/project-dsai-business-case-automated-customer-reviews/data/amazon-customer-reviews/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

# EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28332 entries, 0 to 28331
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   28332 non-null  object 
 1   dateAdded            28332 non-null  object 
 2   dateUpdated          28332 non-null  object 
 3   name                 28332 non-null  object 
 4   asins                28332 non-null  object 
 5   brand                28332 non-null  object 
 6   categories           28332 non-null  object 
 7   primaryCategories    28332 non-null  object 
 8   imageURLs            28332 non-null  object 
 9   keys                 28332 non-null  object 
 10  manufacturer         28332 non-null  object 
 11  manufacturerNumber   28332 non-null  object 
 12  reviews.date         28332 non-null  object 
 13  reviews.dateSeen     28332 non-null  object 
 14  reviews.didPurchase  9 non-null      object 
 15  reviews.doRecommend  16086 non-null 

In [4]:
df.shape

(28332, 24)

In [47]:
df.columns = df.columns.str.lower()

In [6]:
df.head()

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,reviews.didpurchase,reviews.dorecommend,reviews.id,reviews.numhelpful,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [48]:
df.isnull().sum()

id                         0
dateadded                  0
dateupdated                0
name                       0
asins                      0
brand                      0
categories                 0
primarycategories          0
imageurls                  0
keys                       0
manufacturer               0
manufacturernumber         0
reviews.date               0
reviews.dateseen           0
reviews.didpurchase    28323
reviews.dorecommend    12246
reviews.id             28291
reviews.numhelpful     12217
reviews.rating             0
reviews.sourceurls         0
reviews.text               0
reviews.title              0
reviews.username           5
sourceurls                 0
dtype: int64

In [49]:
df = df.drop(columns=["reviews.didpurchase", "reviews.id", "reviews.numhelpful", "reviews.dorecommend"])

In [50]:
df.isnull().sum()

id                    0
dateadded             0
dateupdated           0
name                  0
asins                 0
brand                 0
categories            0
primarycategories     0
imageurls             0
keys                  0
manufacturer          0
manufacturernumber    0
reviews.date          0
reviews.dateseen      0
reviews.rating        0
reviews.sourceurls    0
reviews.text          0
reviews.title         0
reviews.username      5
sourceurls            0
dtype: int64

# Task 1: Review Classification

## Create sentiment categories

In [10]:
df["reviews.rating"].value_counts()

reviews.rating
5    19897
4     5648
3     1206
1      965
2      616
Name: count, dtype: int64

In [11]:
sentiment_map = {1: "negative", 2: "negative", 3: "neutral", 4: "positive", 5: "positive"}
df["rating_sentiment"] = df["reviews.rating"].map(sentiment_map)

In [12]:
df.head()

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,manufacturernumber,reviews.date,reviews.dateseen,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls,rating_sentiment
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2017-03-02T00:00:00.000Z,2017-08-28T00:00:00Z,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...",neutral
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-08-31T00:00:00.000Z,2017-08-28T00:00:00Z,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...",positive
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-12-06T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...",positive
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-03-28T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...",positive
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2017-04-22T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...",positive


In [13]:
df["rating_sentiment"].value_counts()

rating_sentiment
positive    25545
negative     1581
neutral      1206
Name: count, dtype: int64

In [14]:
percentage_of = round((df["rating_sentiment"].value_counts() / len(df) * 100), 2)

percentage_of

# very imbalanced dataset

rating_sentiment
positive    90.16
negative     5.58
neutral      4.26
Name: count, dtype: float64

## Downsample data for balanced classes

In [15]:
# Split the dataframe into three: positive, negative, neutral
positive = (df[df["rating_sentiment"] == "positive"])
negative = (df[df["rating_sentiment"] == "negative"])
neutral =  (df[df["rating_sentiment"] == "neutral"])

positive_sampled = positive.sample(n=1206)
negative_sampled = negative.sample(n=1206)

len(positive_sampled)

# concat into one balanced dataframe 

df_balanced = pd.concat([negative_sampled, positive_sampled, neutral])

# confirm
df_balanced["rating_sentiment"].value_counts()

rating_sentiment
negative    1206
positive    1206
neutral     1206
Name: count, dtype: int64

## Choose a classifier model

- 0: Negative
- 1: Neutral
- 2: Positive

In [16]:
# pick a model 
# CardiffNLP baseline tokenizes the model
classifier = pipeline(task="sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# view a sample 

sample_reviews = df_balanced["reviews.text"].head(10).tolist()

results = classifier(sample_reviews)
results


Device set to use mps:0


[{'label': 'LABEL_0', 'score': 0.8465775847434998},
 {'label': 'LABEL_2', 'score': 0.8215872645378113},
 {'label': 'LABEL_1', 'score': 0.43460822105407715},
 {'label': 'LABEL_0', 'score': 0.92408287525177},
 {'label': 'LABEL_0', 'score': 0.8691161870956421},
 {'label': 'LABEL_0', 'score': 0.9635699987411499},
 {'label': 'LABEL_0', 'score': 0.6331484317779541},
 {'label': 'LABEL_0', 'score': 0.9678872227668762},
 {'label': 'LABEL_0', 'score': 0.5253010988235474},
 {'label': 'LABEL_0', 'score': 0.6237383484840393}]

In [22]:
sample_sentiment = df_balanced["rating_sentiment"].head(10).tolist
sample_sentiment

<bound method IndexOpsMixin.tolist of 22631    negative
11286    negative
7993     negative
20705    negative
8362     negative
24005    negative
11347    negative
16890    negative
6103     negative
18956    negative
Name: rating_sentiment, dtype: object>

In [23]:
full_results = classifier(df_balanced["reviews.text"].tolist(), truncation=True, max_length=512)

In [24]:
label_strings = [r["label"] for r in full_results]
# returns a list of strings
label_strings
# turn the list of strings into a Series using pd.Series
labels_for_mapping = pd.Series(label_strings)
# now labels are ready for mapping
labels_for_mapping


0       LABEL_0
1       LABEL_0
2       LABEL_0
3       LABEL_0
4       LABEL_0
         ...   
3613    LABEL_0
3614    LABEL_0
3615    LABEL_0
3616    LABEL_0
3617    LABEL_2
Length: 3618, dtype: object

In [25]:
# map the labels into sentiment and create a new column based on the predicted sentiment
label_map = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}
df_balanced["predicted_sentiment"] = labels_for_mapping.map(label_map).values

In [26]:
predicted = df_balanced["predicted_sentiment"]
actual = df_balanced["rating_sentiment"]
print(classification_report(actual, predicted))

              precision    recall  f1-score   support

    negative       0.68      0.76      0.72      1206
     neutral       0.53      0.26      0.35      1206
    positive       0.65      0.91      0.76      1206

    accuracy                           0.64      3618
   macro avg       0.62      0.64      0.61      3618
weighted avg       0.62      0.64      0.61      3618



### Export class-balanced dataset to csv for use in Colab

In [27]:
df_balanced.to_csv('balanced_reviews.csv', index=False)

# 2. Product Category Clustering

In [28]:
df["categories"]

0        AA,AAA,Health,Electronics,Health & Household,C...
1        AA,AAA,Health,Electronics,Health & Household,C...
2        AA,AAA,Health,Electronics,Health & Household,C...
3        AA,AAA,Health,Electronics,Health & Household,C...
4        AA,AAA,Health,Electronics,Health & Household,C...
                               ...                        
28327    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28328    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28329    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28330    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28331    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
Name: categories, Length: 28332, dtype: object

In [29]:
df["categories"].value_counts()

categories
AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           8343
AA,AAA,Electronics Features,Health,Electro

## Import local LLM (Qwen2.5) with Ollama for clustering

In [ ]:
# view primary categories as benchmark for meta categories
df["primarycategories"].value_counts()

primarycategories
Electronics                    13995
Health & Beauty                12071
Toys & Games,Electronics        1676
Office Supplies,Electronics      386
Electronics,Media                185
Office Supplies                    9
Animals & Pet Supplies             6
Home & Garden                      2
Electronics,Furniture              2
Name: count, dtype: int64

In [ ]:
# Conduct EDA to understand product categories 

office_supplies = df[df["primarycategories"] == "Office Supplies"]
office_supplies

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,manufacturernumber,reviews.date,reviews.dateseen,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls,rating_sentiment
12156,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-03-02T00:00:00.000Z,"2017-09-21T00:00:00Z,2017-08-26T00:00:00Z,2017...",5,http://www.ebay.com/itm/File-Pocket-Expanding-...,This folder also expands on the bottom. Will a...,Bottom expands as well,happypersonohyeh,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12157,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2019-03-25T00:00:00.000Z,"2019-04-22T00:00:00Z,2019-04-19T00:00:00Z",5,https://www.ebay.com/itm/Expanding-Accordion-F...,Exactly what I needed.,Folder,99kurama99,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12158,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-11-09T00:00:00.000Z,2018-06-20T00:00:00Z,5,https://www.ebay.com/itm/Expanding-File-Tabs-F...,Nice product. Will work perfectly for my proje...,Folder,juleekae,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12159,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-03-27T00:00:00.000Z,"2017-10-30T00:00:00Z,2017-09-21T00:00:00Z,2017...",5,http://www.ebay.com/itm/File-Pocket-Expanding-...,A useful folder organizer with good quality.,Good purchase,ml2shop,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12160,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-01-12T00:00:00.000Z,"2017-09-21T00:00:00Z,2017-08-26T00:00:00Z,2017...",5,http://www.ebay.com/itm/File-Pocket-Expanding-...,I love the portability of this expandable file...,Great product and cost,connieinla,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12161,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2018-06-12T00:00:00.000Z,2018-06-20T00:00:00Z,5,https://www.ebay.com/urw/product-reviews/10014...,Exactly what I needed!!!!,Perfect!,lobo5981,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12162,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder..

In [21]:
df["primarycategories"]

0        Health & Beauty
1        Health & Beauty
2        Health & Beauty
3        Health & Beauty
4        Health & Beauty
              ...       
28327        Electronics
28328        Electronics
28329        Electronics
28330        Electronics
28331        Electronics
Name: primarycategories, Length: 28332, dtype: object

In [51]:
unique_categories = df["categories"].unique()
unique_categories.tolist()

['AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care',
 'Audio & Video Accessories,TV, Video & Home Audio,Home Audio & Video Accessories,DVD Cases,Electronics,Media Cases & Storage,TV, Video & Audio Accessories,Media Storage & Organization,Disc Storage Wallets,Consumer Electronics,CD, DVD & Blu-ray Discs,Blank Audio/Video Media,Accessories & Supplies,all electronics',
 'Stereos,Remote Controls,Audio Docks & Mini Speakers,Kitchen & Dining Features,Home & Tools,Electronics,New Networking,Wireless Speakers,Home, Garage & Office,Portable Speakers,iPod, Audio Player Accessories,Smart Home,Surveillance,Home Improvement,Smart Home & Home Automation Devices,Alarms & Sensors,Amazon Devices,Amazon,Holiday Shop,Kitchen Dining Features,Amazon Echo,Amazon Echo Accessories,Networking,Speaker Sy

In [52]:
category_map = {}

# use qwen to loop through the categories and create a dictionary of categories and primary categories 

for cat in unique_categories:
    response = ollama.chat(model="qwen2.5", messages=[
        {"role": "user", "content": f"""Classify this product category: '{cat}' into ONE of: 'Electronics', 'Health & Beauty', 'Tablets & E-readers', 'Home & Kitchen', 'Office Supplies', 'Pet Supplies'. Reply with ONLY the category name. 
         DO NOT invent your own category name. Use ONLY the names provided.
         If the product category contains Health & Beauty or Health and Beauty, then the product category is: Health & Beauty
         Litter boxes are classified as Pet Supplies.
         
         """}
    ])
    category_map[cat] = response["message"]["content"]
    print(response["message"]["content"])

Health & Beauty
Electronics
Electronics
Electronics
Electronics
Electronics
Health & Beauty
Electronics
Electronics
Electronics
Electronics
Electronics
Office Supplies
Pet Supplies
Electronics
Electronics
Home & Kitchen
Electronics
Electronics
Electronics
Tablets & E-readers
Electronics
Pet Supplies
Electronics
Electronics
Electronics
Tablets & E-readers
Home & Kitchen
Electronics
Tablets & E-readers
Electronics
Electronics
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Pet Supplies
Tablets & E-readers
Pet Supplies
Electronics
Electronics
Tablets & E-readers
Electronics
Electronics
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Electronics
Tablets & E-readers
Electronics
Electronics
Electronics
Electronics
Electronics


In [24]:
# turn the dictionary into a dataframe 
# use .items() to split the key:value pairs into a list of tuples so pd.Dataframe can create two colums, key and value

mapping_df = pd.DataFrame(category_map.items(), columns=["categories", "primary_category"])
mapping_df.head()

,categories,primary_category
0,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty
1,"Audio & Video Accessories,TV, Video & Home Aud...",Electronics
2,"Stereos,Remote Controls,Audio Docks & Mini Spe...",Electronics
3,"Amazon Echo,Virtual Assistant Speakers,Electro...",Electronics
4,"Amazon Echo,Networking,Home & Tools,Smart Home...",Electronics


In [ ]:
df["meta_category"] = df["categories"].map(category_map) # map the LLM's primary categories to the dataframe's categories as "meta_category"

In [26]:

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
df[["categories", "primarycategories", "meta_category"]].value_counts()

categories                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  primarycategories            meta_category      
AA,AAA,Heal

In [27]:
df.columns

Index(['id', 'dateadded', 'dateupdated', 'name', 'asins', 'brand',
       'categories', 'primarycategories', 'imageurls', 'keys', 'manufacturer',
       'manufacturernumber', 'reviews.date', 'reviews.dateseen',
       'reviews.rating', 'reviews.sourceurls', 'reviews.text', 'reviews.title',
       'reviews.username', 'sourceurls', 'rating_sentiment', 'meta_category'],
      dtype='object')

## Save the df with meta_categories and rating_sentiment labels as a csv to save tokens & time when reloading

In [28]:
df.to_csv("amazon_sentiment_categories.csv", index=False)

In [2]:
new_df = pd.read_csv("/Users/chandlershortlidge/Desktop/Ironhack/project-dsai-business-case-automated-customer-reviews/notebooks/amazon_sentiment_categories.csv")

# Task 3: Review Summarization Using Generative AI

In [3]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
new_df.groupby(['meta_category', 'name'])['reviews.rating'].mean().sort_values()

meta_category        name                                                                                                                                                  
Tablets & E-readers  Oem Amazon Kindle Power Usb Adapter Wall Travel Charger Fire/dx/+micro Usb Cable                                                                          1.000000
Electronics          Certified Refurbished Amazon Fire TV with Alexa Voice Remote                                                                                              2.800000
                     Amazon Kindle Replacement Power Adapter (Fits Latest Generation Kindle and Kindle DX) For shipment in the U.S only                                        2.800000
Pet Supplies         AmazonBasics Double-Door Folding Metal Dog Crate - Medium (36x23x25 Inches)                                                                               3.500000
Home & Kitchen       AmazonBasics Silicone Hot Handle Cover/Holder - Red                    

## Electronics

In [4]:
electronics = new_df[new_df["meta_category"] == "Electronics"]


In [72]:
top_3_e = electronics.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='num_reviews', ascending=False).head(3)

top_3_e

avg_rating  \
meta_category name                                                                                            
Electronics   Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers    4.597217   
              Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case               4.531026   
              Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Blue Kid-Proof Case               4.534035   

                                                                                                 num_reviews  
meta_category name                                                                                            
Electronics   Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers         2443  
              Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case                    1676  
              Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Blue Kid-Proof Case                    1425

In [75]:
e_1 = new_df[new_df["name"] == "Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers"]["reviews.text"]
e_1.to_list()

['While there are some apps that are missing due to not using the Google Play Store the list is minimal. The tablet itself is quick, responsive, and noticeable nicer than the 7" fire. Picked this up when it was on sale for black friday sales. It\'s great for in front of the TV and catching up on email and light web browsing.',
 'I bought one last year for one cousin and had to continue for another this year. Great buy imo.',
 'I will try again for the 3rd time with my new Amazon Fire. It seems to have nice upgrades.',
 "bought this for my father, who never did much on computers and was not sure he would actually use this. He unwrapped it, asked for some games to be downloaded and hasn't looked back! He has used the internet as well as the game apps and couldn't be happier.",
 'This was a request from our 7 yr old granddaugter who had always asked her older sister if she could share hers due to we bought this same tablet/size a few yrs earlier. Loved the security/parental control option

In [76]:
e_2 = new_df[new_df["name"] == "Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case"]["reviews.text"]
e_2.to_list()

['great tablet for kids when traveling. my grandchild loves it.',
 'Bought this for my 2 yr old daughter. She was able to use it within minutes. She loves bed it, plays games on it everyday as well as we bought apps to read to her. She loves it all.',
 "Kindle products are fantastic. I've used a kindle & now have a Kindle fire. Use it everyday, so naturally, I would select a Kindle for my granddaughter.",
 "Best buy ever for my 3 year old. Only purchased because they were out of the Galaxy tablet. I'm glad. This tablet is PERFECT for my 3 year old. I love that it has 2 modes. The kids mode sometimes freeze and takes a minutes for apps to open. Also didn't download Netflix movies like indicated. But regular mode works great. My daughter loves watching weird homemade youtube movies anyway.",
 "My little girl loves her Kindle. I'm so happy I bought an upgrade to her previous kindle. Best Christmas gift ever.",
 "My Grandson can't put it down! With access to so many educational apps and ga

In [77]:
e_3 = new_df[new_df["name"] == "Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Blue Kid-Proof Case"]["reviews.text"]
e_3.to_list()

['great tablet for kids when traveling. my grandchild loves it.',
 'Bought this for my 2 yr old daughter. She was able to use it within minutes. She loves bed it, plays games on it everyday as well as we bought apps to read to her. She loves it all.',
 "Kindle products are fantastic. I've used a kindle & now have a Kindle fire. Use it everyday, so naturally, I would select a Kindle for my granddaughter.",
 "My little girl loves her Kindle. I'm so happy I bought an upgrade to her previous kindle. Best Christmas gift ever.",
 'My kids have grown up with technology in the house and their original tablets have started to become outdated. We have since switched to these fire tablets to help extend their gaming. While if they have never had a tablet before the speeds are OK but Im afraid the processing power of these "kids" versions might lag over a short amount of time and they will just become a picture viewer. Time will only tell.',
 "This is the second Kid's version of the Fire that I've

In [69]:
worst_e_product = electronics.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='avg_rating', ascending=True).head(1)

worst_e_product

,,avg_rating,num_reviews
meta_category,name,,
Electronics,Amazon Kindle Replacement Power Adapter (Fits Latest Generation Kindle and Kindle DX) For shipment in the U.S only,2.8,5


In [61]:
electronic_reviews_low = new_df[new_df["name"] == "Amazon Kindle Replacement Power Adapter (Fits Latest Generation Kindle and Kindle DX) For shipment in the U.S only"]["reviews.text"]

In [66]:
electronic_reviews_low.to_list()

['Since the details for the items are a little sparse I thought I would provide the specs listed on the power adapter.AC Input 100-240 Volts, 0.15 Amps, 50/60 HzDC Output 4.9 Volts, 0.85 AmpsThis adapter has a Flextronics part number of 09500043-200, Model A00810',
 'I was initially happy that my Kindle replacement charging cord arrived so promptly, it seemed to work fine. But after only one use, the outer covering of the cord started cracking falling off in bits. It now is almost completely covered by electrical tape, as the bits fall off w/ each every use. I will end up having to buy a new one after all...',
 "I bought the Kindle when I was in the USA but on my return to Australia I discovered I left the power adapter behind in my condo in the USA. For some reason Amazon cannot or will not mail a replacement cable to Australia so now I'm unable to re-charge my Kindle as the Micro USB cables available in Australia do not fit the Kindle as it is tiny compared to the ones available here

In [12]:
summaries = {}

In [ ]:
# goal, give qwen the names, raings, and reviews, then ask



response = ollama.chat(model='qwen2.5', messages=[
{"role": "user", "content": f""" 
 

Write a short article (like a blog post) the product category. The output should include:

Top 3 products {top_3_e} and key differences between them {e_1}, {e_2}, {e_3}.
Top complaints for each of those products.

----- 
You should also include the worst product {worst_e_product} in the category and why it should be avoided {electronic_reviews_low}.
Do not forget to include this.
----

Make sure the style is like a blog.

""" }

])

summaries["electronics"] = response['message']['content']

In [94]:
print(response['message']['content'])

### Top 3 Best-Selling Amazon Fire Tablets and Key Differences

When it comes to finding the perfect tablet for your family or personal use, Amazon's Fire line offers some of the best options on the market. Let’s dive into the top three best-selling products and explore what makes them stand out.

#### 1. **Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine**
   - **Average Rating:** 4.597
   - **Number of Reviews:** 2,443

**Features:**
- Equipped with a large 8-inch HD display.
- Comes pre-installed with Alexa for voice control and smart home integration.
- Offers plenty of storage space (16 GB) to store your favorite apps, games, and media.

**Pros:**
- Quick and responsive performance.
- Great for streaming videos, browsing the web, and managing emails.
- Affordable at just under $100.

**Cons:**
- Some users report minor lag in performance, but this is generally not a significant issue.

#### 2. **Fire Kids Edition Tablet, 7 Display, Wi-Fi, 16 GB, Pink Kid-Proof Case**
  

In [86]:
electronic_blog_summary = summaries["electronics"]

In [ ]:


with open("summaries.json", "w") as f:
    json.dump(summaries, f)

In [91]:
with open("summaries.json", "r") as f:
    summaries = json.load(f)

In [92]:
summaries

{'electronics': "### Top 3 Best-Selling Amazon Fire Tablets and Key Differences\n\nWhen it comes to finding the perfect tablet for your family or personal use, Amazon's Fire line offers some of the best options on the market. Let’s dive into the top three best-selling products and explore what makes them stand out.\n\n#### 1. **Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine**\n   - **Average Rating:** 4.597\n   - **Number of Reviews:** 2,443\n\n**Features:**\n- Equipped with a large 8-inch HD display.\n- Comes pre-installed with Alexa for voice control and smart home integration.\n- Offers plenty of storage space (16 GB) to store your favorite apps, games, and media.\n\n**Pros:**\n- Quick and responsive performance.\n- Great for streaming videos, browsing the web, and managing emails.\n- Affordable at just under $100.\n\n**Cons:**\n- Some users report minor lag in performance, but this is generally not a significant issue.\n\n#### 2. **Fire Kids Edition Tablet, 7 Display, 

## Tablets & E-readers

In [95]:
tablets_ereaders = new_df[new_df["meta_category"] == "Tablets & E-readers"]


In [102]:
top_3_tab_e = tablets_ereaders.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='num_reviews', ascending=False).head(4)

top_3_tab_e


avg_rating  \
meta_category       name                                                                                                                                                       
Tablets & E-readers All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Black                                                           4.584388   
                    Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black                                                                           4.506836   
                    Fire Tablet with Alexa, 7 Display, 16 GB, Blue - with Special Offers                                                                            4.511651   
                    Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers    4.726733   

                                                                                                                                                                  num_reviews  
meta_category       name                                                                                                                                                       
Tablets & E-readers All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Black                                                                2370  
                    Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black                                                                                1024  
                    Fire Tablet with Alexa, 7 Display, 16 GB, Blue - with Special Offers                                                                                  987  
                    Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers          505

In [98]:
tab_e_1 = new_df[new_df["name"] == "All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Black"]["reviews.text"]
tab_e_1.to_list()

['While there are some apps that are missing due to not using the Google Play Store the list is minimal. The tablet itself is quick, responsive, and noticeable nicer than the 7" fire. Picked this up when it was on sale for black friday sales. It\'s great for in front of the TV and catching up on email and light web browsing.',
 'I bought one last year for one cousin and had to continue for another this year. Great buy imo.',
 'I will try again for the 3rd time with my new Amazon Fire. It seems to have nice upgrades.',
 "bought this for my father, who never did much on computers and was not sure he would actually use this. He unwrapped it, asked for some games to be downloaded and hasn't looked back! He has used the internet as well as the game apps and couldn't be happier.",
 'This was a request from our 7 yr old granddaugter who had always asked her older sister if she could share hers due to we bought this same tablet/size a few yrs earlier. Loved the security/parental control option

In [100]:
tab_e_1 = new_df[new_df["name"] == "All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers, Black"]["reviews.text"]
tab_e_1.to_list()

['While there are some apps that are missing due to not using the Google Play Store the list is minimal. The tablet itself is quick, responsive, and noticeable nicer than the 7" fire. Picked this up when it was on sale for black friday sales. It\'s great for in front of the TV and catching up on email and light web browsing.',
 'I bought one last year for one cousin and had to continue for another this year. Great buy imo.',
 'I will try again for the 3rd time with my new Amazon Fire. It seems to have nice upgrades.',
 "bought this for my father, who never did much on computers and was not sure he would actually use this. He unwrapped it, asked for some games to be downloaded and hasn't looked back! He has used the internet as well as the game apps and couldn't be happier.",
 'This was a request from our 7 yr old granddaugter who had always asked her older sister if she could share hers due to we bought this same tablet/size a few yrs earlier. Loved the security/parental control option

In [104]:
tab_e_2 = new_df[new_df["name"] == "Fire Tablet, 7 Display, Wi-Fi, 16 GB - Includes Special Offers, Black"]["reviews.text"]
tab_e_2.to_list()

['Bought this to carry around for Internet and gaming. So far very happy.',
 'First bought the Fire HD 8" for myself and liked it so well that I got this one for my wife.',
 "I'm an author and frequently give these away as promotional prizes. My readers love them.",
 "This was a birthday present for my sister, and she had no trouble getting it set up for use. I was happy to find one with 16GB, not an inadequate 8GB. My other purpose in buying it was to test (on a cheap machine, just in case!) a procedure for installing Google's Play Store. That was a success, and I got a Fire HD 10 for myself.",
 "I bought this as a gift to my parents. The unfortunate part was that I was a little misinformed about the connectivity between the tablet and a chrome cast that I purchases as well as part of the gift. The tablet is not compatible with the table and that was a little disappointing. Other than that, it's an alright product.",
 "This product was purchased for my son to use with another product 

In [106]:
tab_e_3 = new_df[new_df["name"] == "Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Wi-Fi - Includes Special Offers"]["reviews.text"]
tab_e_3.to_list()

['I bought this so i can save all of my books. It was a bit small at first but you get used to it. Overall i am very satistfy',
 'I bought this as a gift. I think it is pretty decent both quality and design.',
 'This is a great e-reader, perhaps a little pricey but well worth the money. It is extremely light and the e-ink is very readable.',
 'I have owned several Kindles over the years, and this one is the best yet. The contrast and clarity is improved over the already-good Paperwhite 2. I like the page turn "buttons" on the bezel as well.',
 "Nothing will replace the joy of flipping through the pages of a book, but living in a studio apartment with my girlfriend, I simply don't have the space for all the books I'd like to own on top of the ones I have now.I got this for my girlfriend because she has difficulty reading on my iPad Air. She has no difficulty using her Kindle Voyage, and she thoroughly enjoys it.I'm actually thinking of getting one for myself farther down the line.",
 'L

In [107]:
worst_tab_e_product = tablets_ereaders.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='avg_rating', ascending=True).head(1)

worst_tab_e_product

,,avg_rating,num_reviews
meta_category,name,,
Tablets & E-readers,Oem Amazon Kindle Power Usb Adapter Wall Travel Charger Fire/dx/+micro Usb Cable,1.0,4


In [112]:
worst_tab_e_reviews = new_df[new_df["name"] == "Oem Amazon Kindle Power Usb Adapter Wall Travel Charger Fire/dx/+micro Usb Cable"]["reviews.text"]
worst_tab_e_reviews.to_list()

['Is Amazon kidding me They want me to pay 19.99 for an item that is a critical part of the functioning of their product Outrageous. I will find another 5V charger.',
 'I bought this thinking that I needed it but I didnt so it was a waste of money. I do so hate to waste money. the Kindle can hook to a regular cell phone charger and it comes with a cable that connects to the computer. For some reason I thought I just had to have this too when I bought my Kindle.',
 "One star because this should've been included when I purchased my Kindles. Bought this second one because I have three paper whites and got tired of playing musical charge chairs. And if Amazon would concentrate on the loyal e-book readers rather than the useless fires and increase memory more than the little bit they did on the latest kindle (I own one) I would be a happy camper. We need the option of adding memory (Amazon invent a chip that only you sell and the only one a kindle can use and I'll purchase). Just give us th

## Tablets & E-readers Summary Blog Output

In [114]:
response = ollama.chat(model='qwen2.5', messages=[
{"role": "user", "content": f""" 
 

Write a short article (like a blog post) the product category. The output should include:

Top 3 products {top_3_tab_e} and key differences between them {tab_e_1}, {tab_e_2}, {tab_e_3}.
Top complaints for each of those products.

----- 
You should also include the worst product {worst_tab_e_product} in the category and why it should be avoided {worst_tab_e_reviews}.
Do not forget to include this.
----

Make sure the style is like a blog.

""" }

])

summaries["tablets_ereaders"] = response['message']['content']
print(response['message']['content'])

### Top 3 Products in Tablets & E-readers: A Reviewer's Guide

When it comes to choosing between tablets and e-readers, Amazon offers a range of options with varying features and performance. Let’s dive into the top three products that reviewers are raving about, along with some key differences and the worst product you should avoid.

#### 1. **All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi, 16 GB - Includes Special Offers**
   - **Average Rating:** 4.58
   - **Number of Reviews:** 2370

This tablet is a fan favorite for its affordability and user-friendly features. Reviewers love it for its quick performance, responsive design, and suitability for tasks like email checking, web browsing, and streaming content. It’s perfect for those who want a reliable device without breaking the bank.

#### 2. **Fire Tablet with Alexa, 7 Display, 16 GB, Blue - with Special Offers**
   - **Average Rating:** 4.51
   - **Number of Reviews:** 987

Equipped with Alexa, this tablet offers an enhanced user e

In [ ]:
with open("summaries.json", "w") as f:
    json.dump(summaries, f)

## Health & Beauty

In [5]:
health_beauty = new_df[new_df["meta_category"] == "Health & Beauty"]

In [6]:
top_3_hb = health_beauty.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='num_reviews', ascending=False).head(3)

top_3_hb

avg_rating  \
meta_category   name                                                                                         
Health & Beauty AmazonBasics AAA Performance Alkaline Batteries (36 Count)                        4.448040   
                AmazonBasics AA Performance Alkaline Batteries (48 Count) - Packaging May Vary    4.453594   

                                                                                                num_reviews  
meta_category   name                                                                                         
Health & Beauty AmazonBasics AAA Performance Alkaline Batteries (36 Count)                             8343  
                AmazonBasics AA Performance Alkaline Batteries (48 Count) - Packaging May Vary         3728

In [7]:
hb_1 = new_df[new_df["name"] == "AmazonBasics AAA Performance Alkaline Batteries (36 Count)"]["reviews.text"]
hb_1.to_list()

['I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.',
 'Bulk is always the less expensive way to go for products like these',
 'Well they are not Duracell but for the price i am happy.',
 'Seem to work as well as name brand batteries at a much better price',
 'These batteries are very long lasting the price is great.',
 "Bought a lot of batteries for Christmas and the AmazonBasics Cell have been good. I haven't noticed a difference between the brand name batteries and the Amazon Basic brand. Just a lot easier to purchase and have arrive at the house and have on hand. Will buy again.",
 'ive not had any problame with these batteries have ordered them in the past been very pleased.',
 'Well if you are looking for cheap non-rechargeable batteries that last quite a while then these are perfect. Nothing more to say.',
 'These do not hold the amount of high power juice like energizer or duracell, but th

In [8]:
hb_2 = new_df[new_df["name"] == "AmazonBasics AA Performance Alkaline Batteries (48 Count) - Packaging May Vary"]["reviews.text"]
hb_2.to_list()

['Bulk is always the less expensive way to go for products like these',
 "Doesn't seem to last as long in some devices as other brands but good for the cost.",
 'ive not had any problame with these batteries have ordered them in the past been very pleased.',
 'Well if you are looking for cheap non-rechargeable batteries that last quite a while then these are perfect. Nothing more to say.',
 'The battery life is much poorer compared to name brands like energizer and Duracell.',
 "These don't last as long as name brand of batteries like Duracell or energizer. They are cheaper but I'd rather pay a little bit more and have them last a lot longer.",
 'I am not a fan of off-brand batteries and so far have been disappointed by off-brand battery performance. I have decided to give these batteries a try and thus far they have performed as expected. At this time I have a number of Amazon batteries because they are low cost.',
 "just got em so I can't really comment on how good the do the job, go

In [9]:
worst_hb_product = health_beauty.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='avg_rating', ascending=True).head(1)

worst_hb_product

,,avg_rating,num_reviews
meta_category,name,,
Health & Beauty,AmazonBasics AAA Performance Alkaline Batteries (36 Count),4.44804,8343


In [10]:
worst_hb_name = worst_hb_product.index.get_level_values('name')[0]
worst_hb_reviews = new_df[new_df["name"] == worst_hb_name]["reviews.text"]
worst_hb_reviews.to_list()

['I order 3 of them and one of the item is bad quality. Is missing backup spring so I have to put a pcs of aluminum to make the battery work.',
 'Bulk is always the less expensive way to go for products like these',
 'Well they are not Duracell but for the price i am happy.',
 'Seem to work as well as name brand batteries at a much better price',
 'These batteries are very long lasting the price is great.',
 "Bought a lot of batteries for Christmas and the AmazonBasics Cell have been good. I haven't noticed a difference between the brand name batteries and the Amazon Basic brand. Just a lot easier to purchase and have arrive at the house and have on hand. Will buy again.",
 'ive not had any problame with these batteries have ordered them in the past been very pleased.',
 'Well if you are looking for cheap non-rechargeable batteries that last quite a while then these are perfect. Nothing more to say.',
 'These do not hold the amount of high power juice like energizer or duracell, but th

### Health & Beauty Summary Blog Output

In [14]:
response = ollama.chat(model='qwen2.5', messages=[
{"role": "user", "content": f""" 
 

Write a short article on the product category {health_beauty}. The output should include:

- Top products {top_3_hb} and key differences between them {hb_1}, {hb_2}.
- Top complaints for each of those products.
- The worst product {worst_hb_product} in the category. DO NOT FORGET TO INCLUDE THIS.
- Discuss why the worst product should be avoided {worst_hb_reviews}.

----

Style: Blog post

""" }

])

summaries["health_beauty"] = response['message']['content']
print(response['message']['content'])

### Are Amazon Basics Batteries Worth the Save?

In today's world of constant gadgets and devices, batteries seem like a commodity we often take for granted. However, when it comes to choosing between name-brand and store-brand alternatives, one option that frequently pops up is Amazon Basics batteries. With their promise of good quality at a lower price point, many wonder if they live up to the hype.

#### The Promise: Good Quality at a Lower Price

One of the main selling points of Amazon Basics batteries is their affordability. For a fraction of the cost of top-name brands like Duracell or Energizer, these alternatives offer a tempting deal that can't be ignored by budget-conscious consumers. But does this lower price come with a trade-off in performance?

#### User Experiences: A Mixed Bag

Let's take a look at some of the experiences shared by users:

- **Positive Reviews:** Many users report that Amazon Basics batteries work just as well as their more expensive counterparts, prov

In [17]:
with open("summaries.json", "w") as f:
    json.dump(summaries, f)

## Home & Kitchen

In [ ]:
home_kitchen = new_df[new_df["meta_category"] == "Home & Kitchen"]

In [20]:
top_3_hk = home_kitchen.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='num_reviews', ascending=False).head(3)

top_3_hk

avg_rating  \
meta_category  name                                                                          
Home & Kitchen AmazonBasics Nespresso Pod Storage Drawer - 50 Capsule Capacity         5.0   
               AmazonBasics Silicone Hot Handle Cover/Holder - Red                     4.0   

                                                                                num_reviews  
meta_category  name                                                                          
Home & Kitchen AmazonBasics Nespresso Pod Storage Drawer - 50 Capsule Capacity            1  
               AmazonBasics Silicone Hot Handle Cover/Holder - Red                        1

In [21]:
hk_1 = new_df[new_df["name"] == "AmazonBasics Nespresso Pod Storage Drawer - 50 Capsule Capacity"]["reviews.text"]
hk_1.to_list()

['I work for Nespresso so I have a lot of capsules. I absolutely LOVE the capsule drawers. So much neater than having the sleeves piled on top of each other.']

In [22]:
hk_2 = new_df[new_df["name"] == "AmazonBasics Silicone Hot Handle Cover/Holder - Red"]["reviews.text"]
hk_2.to_list()

['It slippes around on pan handle']

In [23]:
worst_hk_product = home_kitchen.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='avg_rating', ascending=True).head(1)

worst_hk_product

,,avg_rating,num_reviews
meta_category,name,,
Home & Kitchen,AmazonBasics Silicone Hot Handle Cover/Holder - Red,4.0,1


In [24]:
worst_hk_name = worst_hk_product.index.get_level_values('name')[0]
worst_hk_reviews = new_df[new_df["name"] == worst_hk_name]["reviews.text"]
worst_hk_reviews.to_list()

['It slippes around on pan handle']

### Home & Kitchen Summary Blog Output

In [ ]:
response = ollama.chat(model='qwen2.5', messages=[
{"role": "user", "content": f""" 
 

Write a short article on the product category {home_kitchen}. The output should include:

- Top products {top_3_hk} and key differences between them {hk_1}, {hk_2}.
- Top complaints for each of those products.
- The worst product {worst_hk_product} in the category. 
- Discuss why the worst product should be avoided {worst_hk_reviews}.

----

Style: Blog post

""" }

])

summaries["home_kitchen"] = response['message']['content']
print(response['message']['content'])

### Top Products in Home & Kitchen

When it comes to essential kitchen items, two products stand out in terms of user satisfaction and popularity. Let's dive into the details of these top-selling products:

#### 1. **AmazonBasics Nespresso Pod Storage Drawer - 50 Capsule Capacity**

**Average Rating:** 5.0
**Number of Reviews:** 1

**Overview:**
This storage drawer from AmazonBasics is a favorite among Nespresso users, offering an organized and convenient way to store their coffee capsules. One reviewer, who works for Nespresso, praised it extensively:
> "I work for Nespresso so I have a lot of capsules. I absolutely LOVE the capsule drawers. So much neater than having the sleeves piled on top of each other."

#### 2. **AmazonBasics Silicone Hot Handle Cover/Holder - Red**

**Average Rating:** 4.0
**Number of Reviews:** 1

**Overview:**
This product is designed to protect oven mitts from hot handles, making cooking safer and more comfortable. However, one review highlighted a potential

In [26]:
with open("summaries.json", "w") as f:
    json.dump(summaries, f)

## Office Supplies

In [27]:
office_supplies = new_df[new_df["meta_category"] == "Office Supplies"]

In [28]:
top_3_os = office_supplies.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='num_reviews', ascending=False).head(3)

top_3_os

avg_rating  \
meta_category   name                                                                                          
Office Supplies AmazonBasics Ventilated Adjustable Laptop Stand                                    4.166667   
                Expanding Accordion File Folder Plastic Portable Document Organizer Letter Size    5.000000   

                                                                                                 num_reviews  
meta_category   name                                                                                          
Office Supplies AmazonBasics Ventilated Adjustable Laptop Stand                                           12  
                Expanding Accordion File Folder Plastic Portable Document Organizer Letter Size            9

In [29]:
os_1 = new_df[new_df["name"] == "AmazonBasics Ventilated Adjustable Laptop Stand"]["reviews.text"]
os_1.to_list()

["With an Income Tax Preparation practice I'm on my laptop 10 hours a day entering data. Using the laptop keyboard and mouse pad on the flat and level angle that laptops offer is stressful. An elevated keyboard serves the purpose of 'tilting' the keyboard into a natural position for typing. This AmazonBasic Ventilated and Adjustable laptop stand is perfect and produces the effect of a typewriter slant for those who grew up on IBM Selectrics or Royals. Fully adjustable for just about any angle you are comfortable with. Well made, light weight and it looks as a 'piece of the laptop' in sight. One small warning: the tabs on the front end that keep your laptop from sliding off may be a bit short for some laptops. The tabs rise only .75 of an inch (3/4). My current, and several past laptops, the front sits 1.0 inches (1) at top front but slanting down at .75 of an inch (3/4) inward. The slant keeps the tabs on the stand from holding the laptop in place - causing the laptop to slip down and 

In [30]:
os_2 = new_df[new_df["name"] == "Expanding Accordion File Folder Plastic Portable Document Organizer Letter Size"]["reviews.text"]
os_2.to_list()

['This folder also expands on the bottom. Will allow A LOT more than when the bottom does not expand. Bought a second. I recommended this to my friends.',
 'Exactly what I needed.',
 'Nice product. Will work perfectly for my projects. Nice and sturdy. Thank you!',
 'A useful folder organizer with good quality.',
 'I love the portability of this expandable file good quality, good cost. I recommend',
 'Exactly what I needed!!!!',
 'Good product cheap price! Very happy with my purchase..',
 'A very good document storage grat',
 'Very sturdy and functional.']

In [31]:
worst_os_product = office_supplies.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='avg_rating', ascending=True).head(1)

worst_os_product

,,avg_rating,num_reviews
meta_category,name,,
Office Supplies,AmazonBasics Ventilated Adjustable Laptop Stand,4.166667,12


In [32]:
worst_os_name = worst_os_product.index.get_level_values('name')[0]
worst_os_reviews = new_df[new_df["name"] == worst_os_name]["reviews.text"]
worst_os_reviews.to_list()

["With an Income Tax Preparation practice I'm on my laptop 10 hours a day entering data. Using the laptop keyboard and mouse pad on the flat and level angle that laptops offer is stressful. An elevated keyboard serves the purpose of 'tilting' the keyboard into a natural position for typing. This AmazonBasic Ventilated and Adjustable laptop stand is perfect and produces the effect of a typewriter slant for those who grew up on IBM Selectrics or Royals. Fully adjustable for just about any angle you are comfortable with. Well made, light weight and it looks as a 'piece of the laptop' in sight. One small warning: the tabs on the front end that keep your laptop from sliding off may be a bit short for some laptops. The tabs rise only .75 of an inch (3/4). My current, and several past laptops, the front sits 1.0 inches (1) at top front but slanting down at .75 of an inch (3/4) inward. The slant keeps the tabs on the stand from holding the laptop in place - causing the laptop to slip down and 

### Office Supplies Summary Blog Output

In [33]:
response = ollama.chat(model='qwen2.5', messages=[
{"role": "user", "content": f""" 
 

Write a short article (like a blog post) the product category. The output should include:

Top 3 products {top_3_os} and key differences between them {os_1}, {os_2}.
Top complaints for each of those products.

----- 
You should also include the worst product {worst_os_product} in the category and why it should be avoided {worst_os_reviews}.
Do not forget to include this.
----

Make sure the style is like a blog.

""" }

])

summaries["office_supplies"] = response['message']['content']
print(response['message']['content'])

### The Perfect Typewriter Slant for Your Laptop: A Review of the Amazon Best-Seller

If you're someone who grew up on IBM Selectrics or Royals and craves that typewriter slant while typing, look no further than the Amazon best-selling laptop stand. This piece is a game-changer, providing a comfortable angle adjustment that brings back those nostalgic typing days.

#### Features That Shine

- **Adjustable Angle**: The stand offers multiple angles, making it perfect for achieving your desired typing position.
- **Mesh Cooling**: Keeps your laptop cool by suspending it in the air, ensuring that both your device and your legs remain comfortable during long sessions.
- **Cord Keeper**: A thoughtful addition that keeps cables neatly organized, preventing tangles or unnecessary clutter on your desk.

#### Design and Durability

The stand is made with a lightweight yet sturdy metal frame. It looks like an integral part of the laptop rather than an added accessory. The perforated base adds to 

In [34]:
with open("summaries.json", "w") as f:
    json.dump(summaries, f)

## Pet Supplies

In [35]:
pet_supplies = new_df[new_df["meta_category"] == "Pet Supplies"]

In [36]:
top_3_ps = pet_supplies.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='num_reviews', ascending=False).head(3)

top_3_ps

avg_rating  \
meta_category name                                                                                       
Pet Supplies  AmazonBasics Double-Door Folding Metal Dog Crate - Medium (36x23x25 Inches)          3.5   
              Cat Litter Box Covered Tray Kitten Extra Large Enclosed Hooded Hidden Toilet         5.0   
              AmazonBasics Single-Door Folding Metal Dog Crate - Large (42x28x30 Inches)           5.0   

                                                                                            num_reviews  
meta_category name                                                                                       
Pet Supplies  AmazonBasics Double-Door Folding Metal Dog Crate - Medium (36x23x25 Inches)             2  
              Cat Litter Box Covered Tray Kitten Extra Large Enclosed Hooded Hidden Toilet            2  
              AmazonBasics Single-Door Folding Metal Dog Crate - Large (42x28x30 Inches)              1

In [37]:
ps_1 = new_df[new_df["name"] == "Cat Litter Box Covered Tray Kitten Extra Large Enclosed Hooded Hidden Toilet"]["reviews.text"]
ps_1.to_list()

["I replaced my cat's x-large litter box to this smaller sized litter box... It's easy to clean and takes less kitty litter to fill it which I like",
 'Great price, good value!']

In [38]:
ps_2 = new_df[new_df["name"] == "AmazonBasics Double-Door Folding Metal Dog Crate - Medium (36x23x25 Inches)"]["reviews.text"]
ps_2.to_list()

['I am very pleased with the dog crate. Very sturdy. No damage or flaws. Was better than some others that I have purchased in a retail store.',
 "The price of the item was very good compared to other cages however the item itself came tilted so now my dog has a tilted dog crate. The tilt to the gates are noticeable but not very drastic. For the price I'll live with it."]

In [39]:
ps_3 = new_df[new_df["name"] == "Two Door Top Load Pet Kennel Travel Crate Dog Cat Pet Cage Carrier Box Tray 23\""]["reviews.text"]
ps_3.to_list()

['Sturdy, lightweight, tops lifts for easy access to pet , easy to lean and not bulky. Carrier will easily fit on your lap in the car. A+']

In [40]:
worst_ps_product = pet_supplies.groupby(['meta_category', 'name']).agg(
    avg_rating=('reviews.rating', 'mean'), 
    num_reviews=('reviews.rating', 'count')
).sort_values(by='avg_rating', ascending=True).head(1)

worst_ps_product

,,avg_rating,num_reviews
meta_category,name,,
Pet Supplies,AmazonBasics Double-Door Folding Metal Dog Crate - Medium (36x23x25 Inches),3.5,2


In [41]:
worst_ps_name = worst_ps_product.index.get_level_values('name')[0]
worst_ps_reviews = new_df[new_df["name"] == worst_ps_name]["reviews.text"]
worst_ps_reviews.to_list()

['I am very pleased with the dog crate. Very sturdy. No damage or flaws. Was better than some others that I have purchased in a retail store.',
 "The price of the item was very good compared to other cages however the item itself came tilted so now my dog has a tilted dog crate. The tilt to the gates are noticeable but not very drastic. For the price I'll live with it."]

### Pet Supplies Summary Blog Output

In [42]:
response = ollama.chat(model='qwen2.5', messages=[
{"role": "user", "content": f""" 
 
Write a short article on the product category {pet_supplies}. The output should include:

- Top products {top_3_ps} and key differences between them {ps_1}, {ps_2}, {ps_3}.
- Top complaints for each of those products.
- The worst product {worst_ps_product} in the category. 
- Discuss why the worst product should be avoided {worst_ps_reviews}.

----

Style: Blog post

""" }

])

summaries["pet_supplies"] = response['message']['content']
print(response['message']['content'])

## Top Products in Pet Supplies: A Comprehensive Review

When it comes to pet supplies, finding the right product for your furry friend can be a daunting task. In this blog post, we will delve into the top products in the Pet Supplies category and highlight their key features, reviews, and why they stand out.

### Top Products:

1. **AmazonBasics Double-Door Folding Metal Dog Crate - Medium (36x23x25 Inches)**
   - **Average Rating:** 3.5
   - **Number of Reviews:** 2

   **Key Features:**
   - Sturdy and well-built.
   - No damage or flaws, according to many reviewers.
   - A bit tilted but still acceptable for the price.

2. **Cat Litter Box Covered Tray Kitten Extra Large Enclosed Hooded Hidden Toilet**
   - **Average Rating:** 5.0
   - **Number of Reviews:** 2

   **Key Features:**
   - Easy to clean.
   - Takes less kitty litter, which is a bonus for many cat owners.

3. **AmazonBasics Single-Door Folding Metal Dog Crate - Large (42x28x30 Inches)**
   - **Average Rating:** 5.0
   

In [43]:
with open("summaries.json", "w") as f:
    json.dump(summaries, f)